## 2. SVM (P.105)

1. SVM 은 데이터의 특성 개수가 작아도 복잡한 결정 경계를 만들어 주는 분류 방식입니다.
2. 데이터 전처리(dummy변수, 정규화) 가 필요하며, 매개변수(하이퍼파마미터) 설정에 따라 분류 분석의 성능에 영향이 큼

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [4]:
#NaN 값 많은 deck(열) 삭제
#embarked와 embark_town 열(변수)는 의미가 동일하므로 embark_town 열 삭제
#del df['embark_town']
print(len(df.columns))
ndf = df.drop(['deck', 'embark_town'], axis=1)
print(len(ndf.columns))

#age 변수값이 NaN인 행을 삭제
print(len(ndf))
ndf.dropna(subset=['age'], how='any', axis=0, inplace=True) #how any일경우 하나라도 포함된다면 해당 행(axis =0) 삭제)
print(len(ndf))
print(ndf.describe(include='all'))

#embark 열 NaN값을 승선도시 중 가장 많이 출연한 데이터 값으로 치환
#ndf['embarked'].value_counts(dropna=True).idxmax()
most_freq = ndf['embarked'].value_counts(dropna=True).idxmax()
print(most_freq)

ndf['embarked'].fillna(most_freq, inplace=True)
print(ndf.describe(include='all'))



# 범주형(문자) 데이터를 숫자로 변환 해주어야 함
# survived, pclass, sex, age, sibsp, parch, embarked
# 범주형 데이터를 모델이 인식할 수 있는 숫자로 변환 : one-hot encoding
onehot_sex = pd.get_dummies(ndf['sex'])
print(onehot_sex)

ndf = pd.concat([ndf, onehot_sex], axis=1)
print(ndf)
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town') # embarked 의 컬럼을 각 변수에 따라 town_$로 지정한다.
print(onehot_embarked)
ndf = pd.concat([ndf, onehot_embarked], axis=1) # ndf에 onehot_embarked 결합한다.
print(ndf)




15
13
891
714
          survived      pclass   sex         age       sibsp       parch  \
count   714.000000  714.000000   714  714.000000  714.000000  714.000000   
unique         NaN         NaN     2         NaN         NaN         NaN   
top            NaN         NaN  male         NaN         NaN         NaN   
freq           NaN         NaN   453         NaN         NaN         NaN   
mean      0.406162    2.236695   NaN   29.699118    0.512605    0.431373   
std       0.491460    0.838250   NaN   14.526497    0.929783    0.853289   
min       0.000000    1.000000   NaN    0.420000    0.000000    0.000000   
25%       0.000000    1.000000   NaN   20.125000    0.000000    0.000000   
50%       0.000000    2.000000   NaN   28.000000    0.000000    0.000000   
75%       1.000000    3.000000   NaN   38.000000    1.000000    1.000000   
max       1.000000    3.000000   NaN   80.000000    5.000000    6.000000   

              fare embarked  class  who adult_male alive alone  
count   

In [5]:
# 분류분석에 사용할 변수 선택

X = ndf[['pclass','female', 'male', 'age', 'sibsp', 'parch', 'town_C', 'town_Q', 'town_S']]
Y = ndf['survived']

In [6]:
# train data와 test data 비율 7:3으로 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)
print(X_train.shape)
print(X_test.shape)


(499, 9)
(215, 9)


In [8]:
from sklearn import svm
svm_model = svm.SVC(kernel = 'rbf')  # kernel 종류는 Linear, Polynimial, Sigmoid 등이 있다. 참고주소: https://bskyvision.com/163
svm_model.fit(X_train, Y_train)  #학습, 모델 생성
y_predict = svm_model.predict(X_test)
print(y_predict[0:10])
print(Y_test.values[0:10])

from sklearn import metrics
svm_matrix = metrics.confusion_matrix(Y_test, y_predict)
print(svm_matrix)

svm_report = metrics.classification_report(Y_test, y_predict)
print(svm_report)


[0 0 1 0 1 1 1 1 1 0]
[0 0 1 0 0 1 1 1 0 0]
[[108  17]
 [ 25  65]]
              precision    recall  f1-score   support

           0       0.81      0.86      0.84       125
           1       0.79      0.72      0.76        90

    accuracy                           0.80       215
   macro avg       0.80      0.79      0.80       215
weighted avg       0.80      0.80      0.80       215



C:\Users\student\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
